In [41]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(41)

# Number of samples
n = 10000

# Generate unique applicant IDs
applicant_ids = ["APP" + str(i).zfill(5) for i in range(1, n + 1)]


In [42]:
# -------------------------------
# Applicant Input Dataset
# -------------------------------

# Annual income: Log-normal distribution with parameters set to roughly yield a mean of ~$60K.
# We compute mu such that E[X] = exp(mu + sigma^2/2) ≈ 60000.
sigma_income = 0.4
mu_income = np.log(60000) - (sigma_income**2 / 2)
annual_income = np.random.lognormal(mean=mu_income, sigma=sigma_income, size=n)

# Monthly income for debt calculation
monthly_income = annual_income / 12

# Self-reported debt: 10–30% of monthly income
self_reported_debt = monthly_income * np.random.uniform(0.10, 0.30, size=n)

# Self-reported expenses: Uniform between 0 and 10,000 CAD
self_reported_expenses = np.random.uniform(0, 10000, size=n)

# Requested credit amount: Uniform between 1,000 and 50,000 CAD
requested_amount = np.random.uniform(1000, 50000, size=n)

# Age: Integer between 19 and 100
age = np.random.randint(19, 101, size=n)

# Province: Choose randomly from two provinces ("ON", "BC")
province = np.random.choice(["ON", "BC"], size=n)

# Employment status: Randomly choose between "Full-time", "Part-time", and "Unemployed"
employment_status = np.random.choice(["Full-time", "Part-time", "Unemployed"], size=n)

# Months employed: For unemployed, set to 0; otherwise, random integer between 1 and 600
months_employed = np.array([
    0 if status == "Unemployed" else np.random.randint(1, 601)
    for status in employment_status
])

# Create the applicant input DataFrame
applicant_input_df = pd.DataFrame({
    "applicant_id": applicant_ids,
    "annual_income": annual_income,
    "self_reported_debt": self_reported_debt,
    "self_reported_expenses": self_reported_expenses,
    "requested_amount": requested_amount,
    "age": age,
    "province": province,
    "employment_status": employment_status,
    "months_employed": months_employed
})

applicant_input_df.to_csv('applicant_data.csv', index=False)

In [43]:
# -------------------------------
# Third-Party Credit Dataset
# -------------------------------

# Credit score: Normal distribution with mean 680 and std 100, clipped to a realistic range (300-900)
credit_score = np.random.normal(loc=680, scale=100, size=n)
credit_score = np.clip(credit_score, 300, 900)

# Total credit limit: Correlate with income by choosing a multiplier between 0.5 and 2.0
total_credit_limit = annual_income * np.random.uniform(0.5, 2.0, size=n)

# Credit utilization: Beta distribution scaled to 0-100% (choose parameters to yield a mean near 30%)
credit_utilization = np.random.beta(a=3, b=7, size=n) * 100

# Estimated debt: Calculated as total_credit_limit * (credit_utilization/100) * 0.03
estimated_debt = total_credit_limit * (credit_utilization / 100) * 0.03

# Number of open accounts: Integer between 0 and 20
num_open_accounts = np.random.randint(0, 21, size=n)

# Number of credit inquiries: Integer between 0 and 10
num_credit_inquiries = np.random.randint(0, 11, size=n)

# Payment history: Choose from defined categories, with "On Time" being more likely
payment_history = np.random.choice(
    ["On Time", "Late <30", "Last 30-60", "Late>60"],
    size=n,
    p=[0.7, 0.1, 0.1, 0.1]
)

# Create the third-party credit DataFrame
third_party_df = pd.DataFrame({
    "applicant_id": applicant_ids,
    "credit_score": credit_score,
    "total_credit_limit": total_credit_limit,
    "credit_utilization": credit_utilization,
    "num_open_accounts": num_open_accounts,
    "num_credit_inquiries": num_credit_inquiries,
    "payment_history": payment_history,
    "estimated_debt": estimated_debt
})
third_party_df.to_csv('tp_credit.csv', index=False)

In [44]:
# Merge Datasets on applicant_id
# -------------------------------

merged_df = pd.merge(applicant_input_df, third_party_df, on="applicant_id")

# Calculate and Add DTI
# -------------------------------
# Total monthly debt = self_reported_debt + estimated_debt
# DTI = (self_reported_debt + estimated_debt + (requested_amount * 0.03)) / (annual_income / 12)
merged_df["DTI"] = (merged_df["self_reported_debt"] + merged_df["estimated_debt"] + (merged_df["requested_amount"] * 0.03)) / (merged_df["annual_income"] / 12)


In [45]:
def determine_approval(row):
    # Denial conditions: if any of these are true, deny
    if (row["credit_score"] < 500) or (row["DTI"] > 50) or (row["credit_utilization"] > 80):
        return 0
    # Approval conditions: must meet all these criteria
    if (row["credit_score"] >= 660) and (row["DTI"] <= 40) and (row["payment_history"] == "On Time"):
        return 1
    # Default to deny if criteria not clearly met
    return 0

merged_df["approved"] = merged_df.apply(determine_approval, axis=1)

In [46]:

# -------------------------------
# Compute Credit Limit Based on Provided Rules
# -------------------------------
def compute_credit_limit(row):
    if row["approved"] == 1:
        income = row["annual_income"]
        cs = row["credit_score"]
        utilization = row["credit_utilization"]
        DTI = row["DTI"]
        emp_status = row["employment_status"]
        months_emp = row["months_employed"]
        payment_hist = row["payment_history"]

        # 1. Base Limit Calculation based on Credit Score
        if cs >= 660:
            base_limit = 0.50 * income
        elif cs >= 500:
            base_limit = 0.25 * income
        else:
            base_limit = 0.10 * income

        # 2. Reduce base limit by 20% if credit utilization > 50%
        if utilization > 50:
            base_limit *= 0.8

        # 3. DTI Adjustment
        if DTI <= 0.30:
            dti_factor = 1.0
        elif DTI <= 0.40:
            dti_factor = 0.75
        else:
            dti_factor = 0.5
        adjusted_limit = base_limit * dti_factor

        # 4. Employment Bonus: +10% if Full-time and months_employed ≥ 12
        if emp_status == "Full-time" and months_emp >= 12:
            adjusted_limit *= 1.1

        # 5. Payment Penalty: -50% if payment history is "Late>60"
        if payment_hist == "Late>60":
            adjusted_limit *= 0.5

        # 6. Apply Credit Score Cap
        if cs >= 750:
            cap = 25000
        elif cs >= 660:
            cap = 15000
        elif cs >= 500:
            cap = 10000
        else:
            cap = 5000

        final_limit = min(adjusted_limit, cap)
        return final_limit
    else: 
        return 0

merged_df["credit_limit"] = merged_df.apply(compute_credit_limit, axis=1)

In [47]:

# -------------------------------
# Compute Interest Rate Based on Provided Rules
# -------------------------------
def compute_interest_rate(row):
    prime = 3.0
    # Base interest rate: prime + 1.5%
    base_rate = prime + 1.5  # 4.5% 
    
    # Credit Score Adjustment
    cs = row["credit_score"]
    if cs >= 750:
        cs_adjust = -1.0
    elif cs >= 660:
        cs_adjust = 0.0
    elif cs >= 500:
        cs_adjust = 2.0
    else:
        cs_adjust = 4.0
    
    # DTI Adjustment: Add 1% if DTI > 30%
    dti_adjust = 1.0 if row["DTI"] > 0.30 else 0.0
    
    # Payment History Adjustment: Add 2% if "Late>60"
    pay_adjust = 2.0 if row["payment_history"] == "Late>60" else 0.0
    
    # Open Accounts Penalty: Add 1% if number of open accounts > 5
    open_adjust = 1.0 if row["num_open_accounts"] > 5 else 0.0

    # Compute adjusted rate
    rate = base_rate + cs_adjust + dti_adjust + pay_adjust + open_adjust

    # Ensure minimum rate is at least 3% and maximum rate is 15%
    min_rate = 3 
    max_rate = 15
    final_rate = max(min(rate, max_rate), min_rate)
    return final_rate

merged_df["interest_rate"] = merged_df.apply(compute_interest_rate, axis=1)

In [48]:
# -------------------------------
# Introduce Noise and Missing Data
# -------------------------------

# Introduce noise: For about 7% of entries in selected numerical columns, multiply by a random factor (0.5 to 1.5)
noise_fraction = 0.07
numerical_columns = [
    "annual_income", "self_reported_debt", "self_reported_expenses", 
    "requested_amount", "credit_score", "total_credit_limit", 
    "credit_utilization", "estimated_debt"
]

for col in numerical_columns:
    noise_indices = np.random.choice(merged_df.index, size=int(noise_fraction * len(merged_df)), replace=False)
    noise_multiplier = np.random.uniform(0.5, 1.5, size=len(noise_indices))
    merged_df.loc[noise_indices, col] *= noise_multiplier

# Introduce missing data: Randomly set about 1.5% of each column's entries to NaN
missing_fraction = 0.015
for col in merged_df.columns:
    missing_indices = np.random.choice(merged_df.index, size=int(missing_fraction * len(merged_df)), replace=False)
    merged_df.loc[missing_indices, col] = np.nan


In [49]:
merged_df.head()

,applicant_id,annual_income,self_reported_debt,self_reported_expenses,requested_amount,age,province,employment_status,months_employed,credit_score,total_credit_limit,credit_utilization,num_open_accounts,num_credit_inquiries,payment_history,estimated_debt,DTI,approved,credit_limit,interest_rate
0,APP00001,49702.718659,536.690803,696.576494,48201.458293,44.0,BC,Full-time,282.0,762.506984,96243.138735,24.073824,6.0,5.0,Late>60,695.082106,0.646520,0.0,0.000000,7.5
1,APP00002,57759.265799,904.956521,4680.780392,36704.574102,60.0,ON,Unemployed,0.0,504.128225,38015.731586,28.658793,16.0,4.0,Late>60,326.845497,0.484689,0.0,0.000000,10.5
2,APP00003,61225.005245,672.313284,10351.975031,32569.900660,51.0,ON,Unemployed,0.0,687.028544,78465.243174,34.311307,7.0,3.0,On Time,807.673518,0.481584,1.0,15000.000000,6.5
3,APP00004,38254.629136,773.556508,9808.978559,14815.106947,68.0,ON,Full-time,77.0,771.572334,45526.074871,45.567986,18.0,6.0,Late>60,622.359460,0.577301,0.0,0.000000,7.5
4,APP00005,69491.332067,1260.890909,4843.509163,10049.657603,61.0,BC,Part-time,314.0,770.279548,131466.005326,22.865260,18.0,6.0,On Time,901.801325,0.425523,1.0,17372.833017,5.5


In [50]:
merged_df.describe()

,annual_income,self_reported_debt,self_reported_expenses,requested_amount,age,months_employed,credit_score,total_credit_limit,credit_utilization,num_open_accounts,num_credit_inquiries,estimated_debt,DTI,approved,credit_limit,interest_rate
count,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000,9850.000000
mean,60018.212772,1000.010736,4985.572955,25559.809573,59.233909,199.782335,677.908724,75353.916629,29.958280,9.978985,4.967310,675.226802,0.514869,0.407614,5827.742026,7.025685
std,25377.364189,535.746974,2935.251127,14208.667861,23.837618,199.727193,112.154586,42593.128032,14.014091,6.062847,3.160117,514.072695,0.165308,0.491416,7621.968798,1.585163
min,10318.982794,100.030332,0.736975,1008.697523,19.000000,0.000000,260.337938,8429.899620,1.830925,0.000000,0.000000,17.523415,0.148759,0.000000,0.000000,3.500000
25%,41886.345043,614.800630,2440.156148,13359.642604,38.000000,0.000000,606.506748,44237.435336,19.371667,5.000000,2.000000,317.806836,0.397466,0.000000,0.000000,5.500000
50%,55370.757241,881.454252,4982.753728,25504.272245,59.000000,151.000000,678.838267,66190.288431,28.522008,10.000000,5.000000,539.339583,0.496105,0.000000,0.000000,6.500000
75%,72946.117462,1254.024421,7493.395082,37536.872802,80.000000,371.000000,750.249882,95810.513673,39.012944,15.000000,8.000000,878.393156,0.612175,1.000000,13267.627436,8.500000
max,227326.084751,4695.711952,14340.264503,71196.975475,100.000000,600.000000,1303.949794,318122.623946,96.605035,20.000000,10.000000,4044.776436,1.691600,1.000000,25000.000000,12.500000


In [51]:
# -------------------------------
# Cleaning Dataset
# -------------------------------
# 1. Remove Duplicates
print("Number of duplicate rows before cleaning:", merged_df.duplicated().sum())
merged_df = merged_df.drop_duplicates()
print("Number of duplicate rows after cleaning:", merged_df.duplicated().sum())

# 2. Check Missing Values
missing_counts = merged_df.isnull().sum()
print("Missing values by column before cleaning:")
print(missing_counts)

# 3. Drop Rows with Missing Values
merged_df = merged_df.dropna()
print("\nAfter dropping missing values, cleaned dataset shape:", merged_df.shape)


Number of duplicate rows before cleaning: 0
Number of duplicate rows after cleaning: 0
Missing values by column before cleaning:
applicant_id              150
annual_income             150
self_reported_debt        150
self_reported_expenses    150
requested_amount          150
age                       150
province                  150
employment_status         150
months_employed           150
credit_score              150
total_credit_limit        150
credit_utilization        150
num_open_accounts         150
num_credit_inquiries      150
payment_history           150
estimated_debt            150
DTI                       150
approved                  150
credit_limit              150
interest_rate             150
dtype: int64

After dropping missing values, cleaned dataset shape: (7411, 20)


In [52]:
merged_df.describe()

,annual_income,self_reported_debt,self_reported_expenses,requested_amount,age,months_employed,credit_score,total_credit_limit,credit_utilization,num_open_accounts,num_credit_inquiries,estimated_debt,DTI,approved,credit_limit,interest_rate
count,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000,7411.000000
mean,59965.399717,1000.296999,4965.780968,25508.615784,59.220213,201.100391,677.960533,75193.076509,30.023696,9.960329,4.965996,676.038934,0.514575,0.406153,5788.533243,7.029618
std,25289.961480,536.360077,2946.066167,14242.559599,23.837791,201.431143,112.400451,42624.942986,13.990210,6.046897,3.156756,515.447409,0.164467,0.491147,7609.433921,1.592249
min,10318.982794,139.723988,0.736975,1008.697523,19.000000,0.000000,260.337938,9155.309249,1.830925,0.000000,0.000000,17.523415,0.148759,0.000000,0.000000,3.500000
25%,41960.817535,610.933076,2406.630432,13265.633950,38.000000,0.000000,606.441442,44032.444331,19.558851,5.000000,2.000000,323.119904,0.397287,0.000000,0.000000,5.500000
50%,55363.856649,882.654207,4953.997858,25486.449430,59.000000,151.000000,678.526282,65900.572999,28.647344,10.000000,5.000000,540.092271,0.496069,0.000000,0.000000,6.500000
75%,72707.397010,1256.407810,7487.103386,37399.991314,80.000000,374.000000,750.945200,95710.917603,38.985884,15.000000,8.000000,873.692556,0.612116,1.000000,13198.881133,8.500000
max,205489.546982,4695.711952,14340.264503,71196.975475,100.000000,600.000000,1303.949794,318122.623946,96.605035,20.000000,10.000000,4044.776436,1.489175,1.000000,25000.000000,12.500000


In [53]:
# Drop 500 samples with field approved == 0
# First, filter rows where approved equals 0
non_approved = merged_df[merged_df["approved"] == 0]

drop_indices = non_approved.sample(n=2500, random_state=42).index
merged_df = merged_df.drop(drop_indices)


# Optionally, reset index after dropping rows
merged_df = merged_df.reset_index(drop=True)

# Display the new shape of the merged dataset
print("New dataset shape:", merged_df.shape)


New dataset shape: (4911, 20)


In [54]:
merged_df.to_csv('merged.csv', index=False)
merged_df.describe()

,annual_income,self_reported_debt,self_reported_expenses,requested_amount,age,months_employed,credit_score,total_credit_limit,credit_utilization,num_open_accounts,num_credit_inquiries,estimated_debt,DTI,approved,credit_limit,interest_rate
count,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000,4911.000000
mean,60015.317853,1005.716372,4954.370067,25501.224268,59.482590,201.262065,702.070249,75578.370466,30.055857,9.990022,4.960497,680.036784,0.516339,0.612910,8735.251448,6.584097
std,25503.970022,542.560727,2936.895014,14232.467496,23.862489,201.672425,107.344244,42558.547102,14.139027,6.027238,3.142130,514.576391,0.166024,0.487134,7851.136457,1.493384
min,12763.188903,141.549032,0.736975,1008.697523,19.000000,0.000000,293.008597,9155.309249,1.830925,0.000000,0.000000,17.523415,0.159083,0.000000,0.000000,3.500000
25%,41882.367789,608.403156,2427.893124,13235.947422,39.000000,0.000000,643.841233,44555.402027,19.478917,5.000000,2.000000,322.292518,0.397287,0.000000,0.000000,5.500000
50%,55467.006744,887.063114,4893.271991,25418.389077,60.000000,151.000000,704.871173,66437.750624,28.520696,10.000000,5.000000,548.414962,0.497376,1.000000,10138.792817,6.500000
75%,72805.338406,1263.724410,7458.108495,37377.155366,80.000000,375.000000,765.701562,96051.339085,39.087966,15.000000,8.000000,883.206275,0.616237,1.000000,15000.000000,7.500000
max,203970.748197,4695.711952,14340.264503,71196.975475,100.000000,600.000000,1303.949794,318122.623946,96.605035,20.000000,10.000000,4044.776436,1.489175,1.000000,25000.000000,12.500000
